# Synthesizer Crash Course


### Getting Started

Synthesizer is a Python package designed to create synthetic astronomical observables from both parametric models and hydrodynamical simulations. It is built to be flexible and modular, allowing users to easily customize and extend its functionality.

You should already have Synthesizer installed if you have followed the installation instructions for SBI-Fitter. If not, you can run:

```bash
pip install cosmos-synthesizer
```

### Synthesizer Grids

Synthesizer relies on pre-computed SPS (Stellar Population Synthesis) model grids to generate synthetic observables. The main documentation for these grids can be found [here](https://synthesizer-project.github.io/synthesizer/emission_grids/grids.html). Pre-computed grids are available for download for several popular SPS models, including: BPASS, FSPS, BC03, and Maraston, and are stored in HDF5 format.

Additionally, pre-computed grids have been generated for a variety of IMFs, and have been post-processed to include nebular emission using Cloudy. You should have a `SYNTHESIZER_GRID_DIR` environment variable pointing to the directory where you have stored these grids. 

For the purposes of this crash course, we will use a test grid from BPASS v2.2.1, but the following will work with any of the available grids.

Grids are handled using the `synthesizer.Grid` class, where the `grid_dir` argument tells the code where the desired Grid lives.

In [1]:
from synthesizer import Grid

grid = Grid("test_grid")

FileNotFoundError: [Errno 2] Unable to open file (unable to open file: name = '/Users/user/Documents/PhD/synthesizer/synthesizer/src/synthesizer/data/grids/test_grid.hdf5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)

### unyt

Synthesizer uses the `unyt` package to handle physical units. You can find more information about `unyt` [here](https://unyt.readthedocs.io/en/latest/).

In [ ]:
from unyt import Kelvin, Msun, Myr

### Instruments and Filters


Synthesizer includes a variety of built-in instruments and filters, which can be found in the [documentation](https://synthesizer-project.github.io/synthesizer/observatories/premade_instruments.html). You can also add your own custom filters if needed, and any filter from SVO can be used with the OBSERVATORY/INSTRUMENT.FILTER syntax, e.g. JWST/NIRCam.F356W.

Individual filters are stored in the `Filter` class, and collection of filters are stored in `FilterCollection`.

An `Instrument` stores a `FilterCollection` as well as other information about the instrument, such as its name and the observatory it belongs to.

For this crash course, we will use a premade instrument which contains all the wide filters for JWST NIRCam, which we can plot.

In [ ]:
from synthesizer.instruments.filters import Filter

filter = Filter("JWST/NIRCam.F356W")

from synthesizer.instruments import JWSTNIRCamWide

nircam = JWSTNIRCamWide()

nircam.filters.plot_transmission_curves()

### Creating a Mock Galaxy

Synthesizer has a framework for creating mock galaxies using parametric and particle models. Here wee will focus on the parametric models, but you can find more information about the particle models in the [documentation](https://synthesizer-project.github.io/synthesizer/galaxy_components/particle_parametric.html).

The framework for creating mock galaxies is built around the `Galaxy` class. A `Galaxy` contains a stellar component, which is a `Stars` instance. The `Stars` class uses the SFH and metallicity model, as well as the SPS Grid we created earlier, to generate the stellar population of the galaxy.

In [ ]:
from synthesizer.parametric import Galaxy, Stars

Before we can create a `Stars` instance, we need to define a star formation history (SFH) and a metallicity distribution. Let's start with the SFH.

### 1. Define Star Formation History

Synthesizer includes several built-in star formation history (SFH) models, including commonly used models such as: 
- Constant
- Exponential
- Delayed Exponential
- Lognormal
- Double Power Law

These parametric models are typically defined by 1-3 parameters, which can be specified when creating the SFH instance. You can also create your own custom SFH models by subclassing the `SFH` class.

In this example we will use a simple constant SFH, which is defined by two parameters `min_age` and `max_age`, which define the age range over which the SFH is constant. We do not define a Star Formation Rate (SFR) or total mass here, as the `SFH` will be normalized to the total mass we provide when creating the `Stars` instance.

In [ ]:
from synthesizer.parametric import SFH

sfh_history = SFH.Constant(min_age=0 * Myr, max_age=100 * Myr)

We can plot the SFH using the `plot_sfh` method, and see that it is constant between 0 and 100 Myr.

In [ ]:
sfh_history.plot_sfh(t_range=(0, 2e8))

#### 2. Define Metallicity Distributions

As SPS grids are typically defined over both age and metallicity, we also need to define a metallicity distribution for our stellar population. Synthesizer includes several built-in metallicity distribution models, including:
- Delta Function
- Gaussian

Here we will use a simple delta function, which is defined by a single parameter `metallicity`, which defines the metallicity of all stars in the population. The metallicity here is defined as the mass fraction of metals, so a metallicity of 0.02 corresponds to solar metallicity.

In [ ]:
from synthesizer.parametric import ZDist

metal_dist = ZDist.DeltaConstant(metallicity=0.02)

#### 3. Creating the Galaxy

Now that we have defined the SFH and metallicity distribution, we can create a `Stars` instance. The `Stars` class takes the age and metallicity arrays of the SPS grid, the SFH and metallicity distribution instances, and the total stellar mass of the population as input.


In [ ]:
stellar_component = Stars(
    grid.log10age,
    grid.metallicity,
    sf_hist=sfh_history,
    metal_dist=metal_dist,
    initial_mass=1e10 * Msun,
)

Now we can finally create a `Galaxy` instance, which takes the `Stars` instance as input. The `Galaxy` class can also take additional components, such as gas or a black hole model, but we will not include those in this example. The galaxy also takes a redshift parameter, which is used to calculate the luminosity distance and apply cosmological redshifting to the SED.

In [ ]:
galaxy = Galaxy(stars=stellar_component, redshift=1)

#### 3. Add Emission Model

We now have a galaxy, but we need to define how to generate the SED from the stellar population. This is done using an emission model, which takes the SPS grid and the galaxy as input, and generates the SED.

Emission models follow a tree structure to model the various components of a galaxy SED.

Even for a simple stellar population, we can still choose between several emission models, which include:
- Incident-only (no nebular emission)
- Nebular line emission only
- Nebular continuum only
- Full nebular emission (lines + continuum)
- Intrinsic (nebular + stellar)

When we include modelling of dust attenuation and emission, non-zero escape fractions, and/or AGN emission, the complexity of the emission model increases significantly.

Premade emission models are listed in the [documentation](https://synthesizer-project.github.io/synthesizer/emission_models/premade_models/premade_models.html).

We encourage you to explore the emission models in the documentation, as they are too numerous to list here. In this example we will use a `TotalEmission` model, which includes both stellar and nebular emission, as well as a simple dust attenuation and emission model.


For flexibility, expected emission model components can be set globally on the emission model, or on individual 'emitters', such as `Star` or `Galaxy` instances. For example, we can set the escape fraction of ionizing photons to 0.1 for the entire emission model, or we can set it to 0.2 for just the stellar component of the galaxy.

Below we set the escape fraction to 0.1 for the entire emission model, but if we did not set it here, but instead set it on the `Galaxy` instance, it would override this value.

Before we create our emission model, we need to define our dust attenuation and emission models. Here we use a simple power-law dust attenuation curve, and a single-temperature blackbody for the dust emission. You can find out more information about the wide range of dust models available in the [documentation](https://synthesizer-project.github.io/synthesizer/emission_models/attenuation/attenuation.html).

In [ ]:
from synthesizer.emission_models.attenuation import PowerLaw
from synthesizer.emission_models.dust.emission import Blackbody

dust_curve = PowerLaw(slope=-0.7)
dust_emission_model = Blackbody(temperature=30 * Kelvin)

Now we have everything we need to create our emission model.

In [ ]:
from synthesizer.emission_models import TotalEmission

emission_model = TotalEmission(
    grid=grid, dust_curve=dust_curve, tau_v=0.3, dust_emission_model=dust_emission_model
)

We can plot the emission tree to see the components of the emission model.

In [ ]:
emission_model.plot_emission_tree()

#### 4. Generate observables

So now we have a galaxy, an emission model, and an instrument. We can now put all of this together to generate synthetic observables, such as photometry and spectra.

The easiest observable to generate is the rest-frame SED. This is done using the `get_spectra` method of the emission model, which takes the galaxy as input, and returns the rest-frame wavelength and luminosity arrays.

##### Rest-frame SED

In [ ]:
galaxy.get_spectra(emission_model=emission_model)
galaxy.get_spectra_combined()

The output spectra is a `SED` object, and the generated spectra for the root emission model and the child models are stored in the `galaxy.stars.spectra` dictionary. We can plot the total SED, as well as the individual components, such as the stellar and nebular emission.

In [ ]:
galaxy.plot_spectra(stellar_spectra=True)

##### Observed-frame SED

To calculate observed frame fluxes, we ned to choose a cosmology. Synthesizer uses `astropy.cosmology` for cosmological calculations, and you can choose any of the built-in cosmologies, or define your own. Here we will use the default Planck18 cosmology. We are also choosing the `Inoue2014` IGM model, which is the only IGM model currently implemented in Synthesizer.

In [ ]:
from astropy.cosmology import Planck18 as cosmo

galaxy.get_observed_spectra(cosmo=cosmo)

We can plot the observed-frame SED, which includes the effects of cosmological redshifting and IGM absorption.

In [ ]:
galaxy.plot_observed_spectra(stellar_spectra=True)

##### Photometric Fluxes

Fluxes in the filters of our chosen instrument filters, filter collection or filter can be calculated using the `get_photo_fnu` method of the galaxy or stars, which takes the filter/filter collection as input, and returns a `PhotometryCollection` object containing the fluxes in each filter.

In [ ]:
fluxes = galaxy.stars.get_photo_fnu(filters=nircam.filters)

In [ ]:
print(fluxes["total"])

##### Other Calculations

We can calculate other observables, such as emission line fluxes, and equivalent widths, or metadata such as the surviving stellar mass, mass-weighted age or total ionizing luminosity using the appropriate methods of the `Galaxy` or `Stars` classes. You can find more information about these methods in the [documentation](https://synthesizer-project.github.io/synthesizer/_autosummary/synthesizer.parametric.stars.html#module-synthesizer.parametric.stars).

### Why does this matter?

This framework of grids, galaxies, stars, and emission models allows for a high degree of flexibility in generating synthetic observables. By mixing and matching different components, users can create a wide variety of galaxy models to suit their needs. 

SBI-Fitters library generation tools build on this framework to create large libraries of synthetic observables for use in simulation-based inference. You can learn more about library generation in the next section of the documentation. 